In [1]:
%load_ext tensorboard
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from datetime import datetime

DATADIR = "datasets/design-classics"
LOGDIR = "logs/scalars/"


In [2]:
IMG_SIZE = 128
BATCH_SIZE = 32
COLOR_MODE = "grayscale"

In [3]:
# Add artificial data to set
training_data_generator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    #width_shift_range=0.2,
    #height_shift_range=0.2,
    #fill_mode="nearest",
    #shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

# Split test & train data
training_generator = training_data_generator.flow_from_directory(
    DATADIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    color_mode=COLOR_MODE,
    class_mode="categorical",
    subset="training"
)
validation_generator = training_data_generator.flow_from_directory(
    DATADIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    color_mode=COLOR_MODE,
    class_mode="categorical",
    subset="validation"
)


Found 3540 images belonging to 9 classes.
Found 880 images belonging to 9 classes.


In [4]:
# Attempt to use ResNet50 as a base for the model
#kernel_initializer = tf.keras.initializers.glorot_uniform(seed=1000)
#trained_model = tf.keras.applications.ResNet50(
#    include_top=False,
#    weights="imagenet",
    #alpha=0.5,
#    input_shape=[IMG_SIZE, IMG_SIZE, 3],
#    pooling="max"
#)
#output = Dense(9, activation="softmax", kernel_initializer=kernel_initializer)(trained_model.output)
#model = tf.keras.Model(inputs=trained_model.input, outputs=output)

conv_layers = [3]
layer_sizes = [128]
dense_layers = [1]

# Compensate for imbalanced dataset
unique, counts = np.unique(training_generator.classes, return_counts=True)
class_weights = dict(zip(unique, np.true_divide(counts.sum(), 9*counts)))

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}-DROPOUT-".format(conv_layer, layer_size, dense_layer, datetime.now().strftime("%Y%m%d-%H%M%S"))
            tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=LOGDIR + NAME)
            model = Sequential()
            model.add(Conv2D(layer_size, (3,3), input_shape=(IMG_SIZE, IMG_SIZE, 1 if COLOR_MODE == "grayscale" else 3)))
            model.add(Activation("relu"))
            model.add(MaxPooling2D(pool_size=(2,2)))

            for i in range(conv_layer - 1):
                model.add(Conv2D(layer_size, (3,3)))
                model.add(Activation("relu"))
                model.add(MaxPooling2D(pool_size=(2,2)))
            
            model.add(Flatten())

            for i in range(dense_layer):
                model.add(Dense(256))
                model.add(Activation("relu"))
                model.add(Dropout(0.2))
                
            model.add(Dense(9))
            model.add(Activation("softmax"))
            
            model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer="adam",
              metrics=["accuracy"])
            
            model.fit(training_generator,
                      epochs=15,
                      steps_per_epoch=training_generator.samples // BATCH_SIZE,
                      validation_data=validation_generator,
                      validation_steps=validation_generator.samples // BATCH_SIZE,
                      callbacks=[tensorboard_callback],
                      class_weight=class_weights
                     )


Epoch 1/15
110/110 [==============================] - 127s 1s/step - loss: 2.2207 - accuracy: 0.1382 - val_loss: 1.9033 - val_accuracy: 0.2812
Epoch 2/15
110/110 [==============================] - 119s 1s/step - loss: 1.8957 - accuracy: 0.2870 - val_loss: 1.6138 - val_accuracy: 0.3993
Epoch 3/15
110/110 [==============================] - 123s 1s/step - loss: 1.6323 - accuracy: 0.4311 - val_loss: 1.4067 - val_accuracy: 0.5162
Epoch 4/15
110/110 [==============================] - 123s 1s/step - loss: 1.4641 - accuracy: 0.4881 - val_loss: 1.4194 - val_accuracy: 0.5255
Epoch 5/15
110/110 [==============================] - 128s 1s/step - loss: 1.2986 - accuracy: 0.5500 - val_loss: 1.2153 - val_accuracy: 0.5810
Epoch 6/15
110/110 [==============================] - 123s 1s/step - loss: 1.1574 - accuracy: 0.6013 - val_loss: 1.1007 - val_accuracy: 0.6296
Epoch 7/15
110/110 [==============================] - 121s 1s/step - loss: 1.0717 - accuracy: 0.6466 - val_loss: 0.9612 - val_accuracy: 0.6562

In [5]:

#%tensorboard --logdir logs/scalars

In [6]:
#model.summary()

In [7]:
#model.save("3C128-1D256-15E-GRAY")